# Kurs Geoprocessing mit Python und Opensource Komponenten 2023 ETH Zürich

1. bis 3.3. 2023

## TEIL I
1.3.2023

Umprojizierung Gemeinden von LV03 nach WGS84

In [ ]:
import os
os.system('ogr2ogr -f "Esri Shapefile" -t_srs "EPSG:4326" \
          Data/GEM_SO_WGS84.shp Data/Gemeinden_Solothurn.shp -overwrite')

Umwandlung in KML:

In [ ]:
import os
os.system('ogr2ogr -f "KML" Data/GEM_SO.kml Data/Gemeinden_Solothurn.shp')

# Arbeiten mit Vektordaten

In [ ]:
from osgeo import ogr
import os

drv = ogr.GetDriverByName("Esri Shapefile")
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
print(path2ds)

datasource = drv.Open(path2ds)
layer = datasource.GetLayer()
#print(layer)

numFtrs = layer.GetFeatureCount()
print(f"Anzahl Features: {numFtrs}")


**so nicht!!**:

fuerSaraina = ogr.GetDriverByName("Esri Shapefile").Open(os.path.join("Data","Gemeinden_Solothurn.shp")).GetLayer(0).GetFeatureCount()
print(fuerSaraina)

In [ ]:
layerDef = layer.GetLayerDefn()
numAtts = layerDef.GetFieldCount()
print(f"Anzahl Attribute: {numAtts}")


In [ ]:
for i in range(numAtts):
    fieldDef = layerDef.GetFieldDefn(i)
    print(f"Attributname: {fieldDef.GetName()}, Typ: {fieldDef.GetType()}")

In [ ]:
spatialRef = layer.GetSpatialRef()
#print(f"Layer '{layer.GetName()}' hat das SRS: \n{spatialRef}")
print("Layer '%s' hat das SRS: \n%s mit %s Features." %(layer.GetName(),spatialRef,numFtrs) )

In [ ]:
#Raeumliche Ausdehnung:
extent = layer.GetExtent()
print(extent)

In [ ]:
ulx = extent[0]
uly = extent[2]
orx = extent[1]
ory = extent[3]
print(f"Ausdehnung des Datensatzes '{layer.GetName()}':\nUnten links: ({ulx},{uly}) und oben rechts: ({orx},{ory})")

In [ ]:
print('Unten-links:', extent[0], extent[2])
print('Oben-rechts:', extent[1], extent[3])

GetGeomType()

In [ ]:
GeomType = ogr.GeometryTypeToName(layer.GetGeomType())
print("Geometrietyp: %s" %GeomType)

Analyse Weltdatensatz **"TM_WORLD_BORDERS-0.3.shp"**

In [ ]:
from osgeo import ogr
import os 

def metadataFromGisLayer(lyr,printOption=False):
    drv = ogr.GetDriverByName("ESRI Shapefile")
    path2ds = lyr
    datasource = drv.Open(path2ds)

    mylayer = datasource.GetLayer(0)
    print(f"Nun gesetzt: {mylayer.GetName()}")

    ftrcnt = mylayer.GetFeatureCount()
    mylayerDef = mylayer.GetLayerDefn()
    numatts = mylayerDef.GetFieldCount()
    if printOption:
        print("Der Datensatz '%s' hat %i Länder mit je %i Attributen" %(mylayer.GetName(),int(ftrcnt),int(numatts)))
    for i in range(numatts):
        fieldDef = mylayerDef.GetFieldDefn(i)
        if printOption:
            print("Attribut  %s ist vom Typ %s" %(fieldDef.GetName(),fieldDef.GetType()))
    sRs = mylayer.GetSpatialRef()
    if printOption:
        print("Layer %s hat folgendes Räumliches Bezugssytem: %s" %(mylayer.GetName(),sRs))

    extent = mylayer.GetExtent()
    if printOption:
        print(extent)
    ulx = round(float(extent[0]),4)
    uly = round(float(extent[2]),4)
    orx = round(float(extent[1]),4)
    ory = round(float(extent[3]),4)
    if printOption:
        print(f"Ausdehnung des Datensatzes '{mylayer.GetName()}':\nUnten links: ({ulx},{uly}) und oben rechts: ({orx},{ory})")

        
        

In [ ]:
myLyr = "Data/TM_WORLD_BORDERS-0.3.shp"
lyrInstance = metadataFromGisLayer(myLyr)

## TEIL II
2.3.2023
<p>Hilfreich: <a href='https://pcjericks.github.io/py-gdalogr-cookbook/layers.html'>https://pcjericks.github.io/py-gdalogr-cookbook/layers.html</a>

In [ ]:
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
lyrInstance = metadataFromGisLayer(path2ds)

In [ ]:
from osgeo import ogr, os

drv = ogr.GetDriverByName("ESRI Shapefile")
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
datasource = drv.Open(path2ds)
mylayer = datasource.GetLayer(0)

feature = mylayer.GetFeature(12)
attributes = feature.items()
for key,value in attributes.items():
    print(f"{key}: {value}")

Geometrieinformation über: GetGeometryRef()

In [ ]:
myGeometry = feature.GetGeometryRef()
#print("Geometrietyp: %s" %geometry)
print(myGeometry.GetGeometryName())
print(myGeometry.GetPointCount())
print(myGeometry.GetGeometryCount())

In [ ]:
def analyzeGeometry(geometry, indent=0):
    s = []
    s.append("  " * indent)
    s.append(geometry.GetGeometryName())
    if geometry.GetPointCount() > 0:
        s.append(" mit %d Stuetzpunkten" % geometry.GetPointCount())
    if geometry.GetGeometryCount() > 0:
        s.append(" enthaelt:")

    print ("".join(s))

    for i in range(geometry.GetGeometryCount()):
        analyzeGeometry(geometry.GetGeometryRef(i), indent+1)


In [ ]:
analyzeGeometry(myGeometry)

In [ ]:
from osgeo import ogr, os

drv = ogr.GetDriverByName("ESRI Shapefile")
path2ds = "Data/TM_WORLD_BORDERS-0.3.shp"
datasource = drv.Open(path2ds)
mylayer = datasource.GetLayer(0)

feature = mylayer.GetFeature(0)
attributes = feature.items()
for key,value in attributes.items():
    print(f"{key}: {value}")
myGeometry = feature.GetGeometryRef()
analyzeGeometry(myGeometry)

*Putting it all together...*

In [ ]:
def featureInfo(lyr,placeName=None,printOption=False):
    drv = ogr.GetDriverByName("ESRI Shapefile")
    path2ds = lyr
    datasource = drv.Open(path2ds)
    mylayer = datasource.GetLayer(0)
    if printOption:
        print(f"Nun gesetzt: {mylayer.GetName()}")    

    if not placeName is None:
        #Attributfilter    
        mylayer.SetAttributeFilter("NAME = '%s'" %placeName)
    
    feature = mylayer.GetNextFeature()
    while feature:
        attributes = feature.items()
        for key,value in attributes.items():
            if printOption:
                print("  %s = %s" % (key, value))
        if printOption:
            print()

        #Geometrietyp:
        gtype = feature.GetGeometryRef().GetGeometryName()
        if printOption:
            print("Geometrietyp: %s" %gtype)

        #Geometrie:
        geometry = feature.GetGeometryRef()
        if printOption:
            print("")
        #print "Geometrietyp: %s" %geometry
        analyzeGeometry(geometry)
        if printOption:
            print("")
        feature = mylayer.GetNextFeature()    

In [ ]:
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
#metadataFromGisLayer(path2ds)
myPlace2Look4 = None
myPlace = input()
if myPlace != '':
    myPlace2Look4 =myPlace
featureInfo(path2ds,myPlace2Look4,True)

In [ ]:
path2ds = "Data/TM_WORLD_BORDERS-0.3.shp"
metadataFromGisLayer(path2ds)
featureInfo(path2ds,None,True)

*Umprojektion*

In [22]:
from osgeo import ogr 
from osgeo import osr  
source = osr.SpatialReference() 
source.ImportFromEPSG(2927)  

target = osr.SpatialReference() 
target.ImportFromEPSG(4326)  

transform = osr.CoordinateTransformation(source, target)  

origpnt = "POINT (1120351.57 741921.42)"
transfpoint = ogr.CreateGeometryFromWkt(origpnt) 
transfpoint.Transform(transform) 
print("Ausgangskoordinaten: %s" %origpnt)
print("Transformierte Koordinaten: %s" %transfpoint.ExportToWkt())

Ausgangskoordinaten: POINT (1120351.57 741921.42)
Transformierte Koordinaten: POINT (47.3488070138318 -122.598149943144)


In [24]:
source = osr.SpatialReference() 
source.ImportFromEPSG(21781)  
transform = osr.CoordinateTransformation(source, target)  

origpnt = "POINT (600000.00 200000.00)"
transfpoint = ogr.CreateGeometryFromWkt(origpnt) 
transfpoint.Transform(transform) 
print("Ausgangskoordinaten: %s" %origpnt)
print("Transformierte Koordinaten: %s" %transfpoint.ExportToWkt())

Ausgangskoordinaten: POINT (600000.00 200000.00)
Transformierte Koordinaten: POINT (46.9510827718711 7.43863242087181)


In [46]:
def reProj(pnt,s,d):
    source = osr.SpatialReference() 
    source.ImportFromEPSG(s)  
    target = osr.SpatialReference() 
    target.ImportFromEPSG(d) 
    transform = osr.CoordinateTransformation(source, target)  
    transfpoint = ogr.CreateGeometryFromWkt(pnt) 
    transfpoint.Transform(transform) 
    return transfpoint.ExportToWkt()

origPnt = "POINT (600000.00 200000.00)"  
reProjPnt = reProj(origPnt,21781,4326).split(" ")


reProjPntX = reProjPnt[1].split("(")[1]
reProjPntY = reProjPnt[2].split(")")[0]
print(f"Transformierte Koordinaten: x={reProjPntX}, y={reProjPntY}")

Transformierte Koordinaten: x=46.9510827718711, y=7.43863242087181


# Umgang mit Rasterdaten

In [47]:
from osgeo import gdal

fn = 'Data/ortho14_5m_rgb_solothurn.tif'
ds = gdal.Open(fn)
if ds is None:
    print ('Datensatz %s konnte nicht geöffnet werden!' %fn)
    sys.exit(1)

#Dimension des Rasterbildes
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount

print ("Anzahl Spalten: %d" %cols)
print ("Anzahl Zeilen: %d" %rows)
print ("Anzahl Baender: %d" %bands)

Anzahl Spalten: 5800
Anzahl Zeilen: 4800
Anzahl Baender: 3


In [49]:
gTr = ds.GetGeoTransform()
print(gTr)

(592000.0, 5.0, 0.0, 237000.0, 0.0, -5.0)


In [50]:
geotransform = ds.GetGeoTransform()
if not geotransform is None:
    print ('Origin = (',geotransform[0], ',',geotransform[3],')')
    print ('Pixel Size = (',geotransform[1], ',',geotransform[5],')')
    print ('Rotation: ',geotransform[2], ' / ',geotransform[4],'')


Origin = ( 592000.0 , 237000.0 )
Pixel Size = ( 5.0 , -5.0 )
Rotation:  0.0  /  0.0 


In [52]:
from osgeo import gdal
import sys

def rasterInfo(rD,outFl,printOption=False):
    ds = gdal.Open(rD)
    if ds is None:
        print ("Datensatz '%s' konnte nicht geöffnet werden!" %rD)
        sys.exit()

    cols = ds.RasterXSize
    rows = ds.RasterYSize
    bands = ds.RasterCount
    if printOption:
        outTxt = f"'{rD}' hat {cols} Spalten, {rows} Zeilen \n und Anzahl Bänder: {bands}."
        with open(outFl,'a') as outF:
            outF.write(outTxt)
        print(outTxt)

*Iteriere über alle Dateien von einem bestimmten Typ in einem Verzeichnis (inkl Unterverzeichnisse)*:

In [53]:
for root, dirs, files in os.walk(r'X:\_GeoProc2023\Data'):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            rasterInfo(os.path.join(root, file),True)

'X:\_GeoProc2023\Data\Elevation_raster.tif' hat 901 Spalten, 901 Zeilen 
         und Anzahl Bänder: 1.
'X:\_GeoProc2023\Data\Ele_slope.tif' hat 901 Spalten, 901 Zeilen 
         und Anzahl Bänder: 1.
'X:\_GeoProc2023\Data\geoworld.tif' hat 2048 Spalten, 1024 Zeilen 
         und Anzahl Bänder: 3.
'X:\_GeoProc2023\Data\ortho14_5m_rgb_solothurn.tif' hat 5800 Spalten, 4800 Zeilen 
         und Anzahl Bänder: 3.


*Ausdehnung Raster*

In [55]:
geotransform = ds.GetGeoTransform()
if not geotransform is None:
    print ('Origin = (',geotransform[0], ',',geotransform[3],')')
    print ('Pixel Size = (',geotransform[1], ',',geotransform[5],')')
    print ('Rotation: ',geotransform[2], ' / ',geotransform[4],'')


Origin = ( 592000.0 , 237000.0 )
Pixel Size = ( 5.0 , -5.0 )
Rotation:  0.0  /  0.0 


*Abfrage von radiometrischen Werten eines Bildes anhand von Koordinaten*

In [54]:
# obtained from http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf and adapted
# script to get pixel values at a set of coordinate by reading in one pixel at a time

import os, sys, numpy, time
from osgeo import gdal
from osgeo.gdalconst import *

# start timing
startTime = time.time()
# coordinates to get pixel values for
xValues = [594000.0, 604000.0, 613500.0,599594.0]
yValues = [229500.0, 231000.0, 222800.0,226081.0]

# register all of the drivers
gdal.AllRegister()
# open the image
ds = gdal.Open('Data/ortho14_5m_rgb_solothurn.tif', GA_ReadOnly)
if ds is None:
    print ('Could not open image')
    sys.exit()

# get image size
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]
print('X | Y | xOffset | yOffset | Wert Band 1 | Wert Band 2 | Wert Band 3 ')

# loop through the coordinates
#for i in range(3):
for i in range(len(xValues)):
    # get x,y
    x = xValues[i]
    y = yValues[i]
    # compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)
    # create a string to print out
    s = str(x) + ' ' + str(y) + ' ' + str(xOffset) + ' ' + str(yOffset) + ' '
    # loop through the bands
    for j in range(bands):
        band = ds.GetRasterBand(j+1) # 1-based index
        #read data and add the value to the string
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0,0]
        #print(data)
        #value2 = numpy.median(data)
        s = s + str(value) + ' '

    #print out the data string
    print(s)
# figure out how long the script took to run
endTime = time.time()
print()
print ('The script took %.3f seconds' %(endTime - startTime))

X | Y | xOffset | yOffset | Wert Band 1 | Wert Band 2 | Wert Band 3 
594000.0 229500.0 400 1500 17 18 22 
604000.0 231000.0 2400 1200 178 162 148 
613500.0 222800.0 4300 2840 73 97 82 
599594.0 226081.0 1518 2183 89 91 85 

The script took 0.027 seconds


In [57]:
# obtained from http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf and adapted
# script to get pixel values at a set of coordinate by reading in one pixel at a time

import os, sys, numpy, time
from osgeo import gdal
from osgeo.gdalconst import *

# start timing
startTime = time.time()
# coordinates to get pixel values for
xValues = [10.1]
yValues = [47.3]

# register all of the drivers
gdal.AllRegister()
# open the image
ds = gdal.Open('Data/Elevation_raster.tif', GA_ReadOnly)
if ds is None:
    print ('Could not open image')
    sys.exit()

# get image size
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]
print('X | Y | xOffset | yOffset | Höhenwert')

# loop through the coordinates
#for i in range(3):
for i in range(len(xValues)):
    # get x,y
    x = xValues[i]
    y = yValues[i]
    # compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)
    # create a string to print out
    s = str(x) + ' ' + str(y) + ' ' + str(xOffset) + ' ' + str(yOffset) + ' '
    # loop through the bands
    for j in range(bands):
        band = ds.GetRasterBand(j+1) # 1-based index
        #read data and add the value to the string
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0,0]
        #print(data)
        #value2 = numpy.median(data)
        s = s + str(value) + ' '

    #print out the data string
    print(s)
# figure out how long the script took to run
endTime = time.time()
print()
print ('The script took %.3f seconds' %(endTime - startTime))

X | Y | xOffset | yOffset | Höhenwert
10.1 47.3 360 720 1571 

The script took 0.005 seconds


In [59]:
#obtained from http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf and adapted

# script to get pixel values at a set of coordinate
# by reading in one pixel at a time
# modified to read altitude, aspect and sloap from the band

import os, sys, numpy, time
from osgeo import gdal
from osgeo.gdalconst import *
import random

random.seed()

'''
Infos zur Rasterdatei:
Lower Left  (   9.9998611,  47.2498611) (  9d59'59.50"E, 47d14'59.50"N)
Upper Right (  10.2501389,  47.5001389) ( 10d15' 0.50"E, 47d30' 0.50"N)
Pixel Size = (0.000277777777778,-0.000277777777778)
Size is 901, 901
'''
xValues = []
yValues = []
pntnumber = input("Anzahl zuf. Punkte:")
for i in range(1, int(pntnumber)+1):
    #Erzeuge zufällige Koordinatenwerte
    rechts = random.random() * 0.000277777777778 * 901 + 9.9998611
    hoch = random.random() * 0.000277777777778 * 901 + 47.2498611

    # coordinates to get pixel values for
    xValues.append(rechts)
    yValues.append(hoch)

    #print("%i. Punkt: x= %f | y= %f" %(i,rechts, hoch))

# start timing
startTime = time.time()

# register all of the drivers
gdal.AllRegister()

imagefilename = "Data/Elevation_raster.tif"
path = "Data/"

# open the image
ds = gdal.Open(imagefilename) #The band is altitude values

if ds is None:
    print ('Could not open image')
    sys.exit(1)

# Translate to slope and aspect /          #gdaldem hillshade input_dem output_hillshade

# File to store results and open in QGIS
rndpntfl = open("_randompointfilewithDEMinfo.csv","w")

sloapcommand = 'gdaldem slope %s %sEle_slope.tif -s 10000' %(imagefilename,path)
#print ("command to run: %s" %sloapcommand)   #gdaldem slope input_dem output_slope_map

os.system(sloapcommand)

aspectcommand = 'gdaldem aspect %s %sEle_aspect.tif' %(imagefilename,path)
#print ("command to run: %s" %aspectcommand)  #gdaldem aspect input_dem output_aspect_map

os.system(aspectcommand)

da= gdal.Open(path + 'Ele_slope.tif')
dl= gdal.Open(path + 'Ele_aspect.tif')

# get image size
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount

bandsaspect = da.RasterCount
bandsslope = dl.RasterCount

# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]


print ('Ursprung der Rasterdatei: ', xOrigin , yOrigin)
print("")
print ('X | Y | xOffset | yOffset | Altitude  | Aspect  | Slope |' )
rndpntfl.write('X, Y, xOffset, yOffset, Altitude, Aspect, Slope \n')

# loop through the coordinates
for k in range(len(xValues)):
    x = xValues[k]
    y = yValues[k]

# compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)

# create a string to print out
    s = str(x) + ' ' + str(y) + ' ' + str(xOffset) + ' ' + str(yOffset) + ' '

# loop through the bands

    for j in range(bands):
        band = ds.GetRasterBand(j+1) # 1-based index
        # read data and add the value to the string
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0,0]
        s = s + str(value) + ' '

    #get aspect value
    for f in range(bandsaspect):
        band_da = da.GetRasterBand(f+1) # 1-based index
        # read data and add the value to the string
        data_da = band_da.ReadAsArray(xOffset, yOffset, 1, 1)
        value_da = data_da[0,0]
        s = s + str(value_da) + ' '

    #get slope value
    for e in range(bandsslope):
        band_dl = dl.GetRasterBand(e+1) # 1-based index

        # read data and add the value to the string
        data_dl = band_dl.ReadAsArray(xOffset, yOffset, 1, 1)
        value_dl = data_dl[0,0]

        s = s + str(value_dl) + ' \n'

    #print out the data string
    s2=s.replace("\n","")
    print (s2)
    rndpntfl.write(s.replace(" ",", "))
    #print ("")


# figure out how long the script took to run
endTime = time.time()
print ('The script took ' + str(endTime - startTime) + ' seconds')
rndpntfl.close()


Anzahl zuf. Punkte:100
Ursprung der Rasterdatei:  9.99986111111111 47.500138888888884

X | Y | xOffset | yOffset | Altitude  | Aspect  | Slope |
10.051171395140079 47.34141061119083 184 571 1542 84.716576 174.02844 
10.061898621575542 47.482500682652855 223 63 1220 55.759758 229.96974 
10.204270252948648 47.33261921729409 735 603 1947 68.33675 204.27444 
10.106075456331256 47.28405589061452 382 777 1691 82.56022 23.552261 
10.110439282615712 47.41042498808234 398 322 1410 69.42876 184.844 
10.197860947344802 47.257292915584465 712 874 1420 75.104965 11.040939 
10.05924197167107 47.40902741179211 213 328 1076 74.55152 21.139824 
10.056822775587321 47.34393656011366 205 562 1517 83.11055 304.81335 
10.232605851986325 47.345336179861455 837 557 1585 77.4383 153.17802 
10.093140526605588 47.47839089462378 335 78 1526 84.731155 341.11392 
10.02095141847199 47.450857694301476 75 177 1062 79.67815 179.0608 
10.159713479769685 47.278483573245346 575 797 1994 76.51231 302.66092 
10.021164472628